# SIADS 516: Homework 1
Version 1.0.20200221.1
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

### Our first mrjob script

Recall the following example from the lectures:

Note the use of the magic command ```%%file```.  You can use this to write the contents of a cell out to a file, which is what we need to do to use mrjob:

In [1]:
%%file word_count.py
from mrjob.job import MRJob
import re

class MRWordFrequencyCount(MRJob):

  ### input: self, in_key, in_value
  def mapper(self, _, line):
    yield "chars", len(line)
    yield "words", len(line.split())
    yield "lines", 1

  ### input: self, in_key from mapper, in_value from mapper
  def reducer(self, key, values):
    yield key, sum(values)
if __name__ == "__main__":
    MRWordFrequencyCount.run()

Overwriting word_count.py


### <font color="magenta">Q1: Explain what each of the yield statements in the above script do.  Provide a list of what the first few iterations through the mapper() step would yield if the script was run against the ```data/gutenberg/short.t1.txt``` file.

1. The first yield statement within the mapper function will, for each iteration through the text file, yield a tuple consisting of the key "chars", followed by the number of characters contained in the line. Whitespace and punctuation are counted as a character.
2. The second yield statement within the mapper function will, for each iteration through the text file, yield a tuple consisting of the key "words", followed by the number of words contained in the line. This is obtained by utilizing the python String split() method, which will split a string into a list of elements(any given line in the txt file) and will, by default, seperate each element by whitespace. One can then take the len() of that list to obtain the number of words in the string. 
3. The third yield statement within the mapper function will, for each iteration through the text file, yield a tuple consisting of the key "lines", followed by the number of lines that was passed in. Since the mapper function consumes one line each iteration, this value can just be hardcoded as one.

Line 1:Project Gutenberg's In the Year 2889, by Jules Verne and Michel Verne

Line 2:

Line 3: This eBook is for the use of anyone anywhere at no cost and with

The output yielded for the first three lines for short.t1.txt would look like so:

**Line 1**
"chars" 69
"words" 12
"lines" 1

**Line 2**
"chars" 0
"words" 0
"lines" 1

**Line 3**
"chars" 64
"words" 14
"lines" 1


Now let's look at the output of running the script against that same file:

In [2]:
!python word_count.py data/gutenberg/short.t1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/word_count.jovyan.20200705.224821.096617
Running step 1 of 1...
job output is in /tmp/word_count.jovyan.20200705.224821.096617/output
Streaming final output from /tmp/word_count.jovyan.20200705.224821.096617/output...
"chars"	10653
"words"	1822
"lines"	200
Removing temp directory /tmp/word_count.jovyan.20200705.224821.096617...


### <font color="magenta">Q2.  Repeat the above cell using the the works of William Shakespeare text file (data/gutenberg/t8.shakespeare.txt).  Provide an interpretation of the output (don't overthink this -- just demonstrate that you can find the relevant information in the output).</font>

In [3]:
!python word_count.py data/gutenberg/t8.shakespeare.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/word_count.jovyan.20200705.224821.580163
Running step 1 of 1...
job output is in /tmp/word_count.jovyan.20200705.224821.580163/output
Streaming final output from /tmp/word_count.jovyan.20200705.224821.580163/output...
"chars"	5333743
"words"	901325
"lines"	124456
Removing temp directory /tmp/word_count.jovyan.20200705.224821.580163...


Since we are not launching our map reduce job onto a specialized platform like Amazon EMR or Cloudera, we do not have to specify any configuration values(for instance, EMR may require a secret access key) - hence, the messages indicating "No configs found." Upon running word_count.py, a temporary directory is created called tmp and creates an empty file called word_count.jovyan.20200703.202608.969845. Then, the intermediary mapper and final key, sum(values) are written to this file, and the output is printed to the console. For the final step, the temporary directory is destroyed. 

### Now let's look at a slightly more complicated example:

In [4]:
%%file most_used_word.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+") # any whitespace or apostrophe, used to split lines below


class MRMostUsedWord(MRJob):
    STOPWORDS = {'i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            if word.lower() not in self.STOPWORDS:
                yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # optimization: sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is used so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)



if __name__ == '__main__':
    import time
    start = time.time()
    MRMostUsedWord.run()
    end = time.time()
    print(end - start)

Overwriting most_used_word.py


### <font color="magenta">Q3: Explain what the yield statements in the  above script do.  Provide a list of what the first few iterations through the steps would yield.

**mapper_get_words()** - The yield statement in this function yields a tuple of each word in the given line, converted to lowercase, and the count, hardcoded as one(potential duplicate words will be summed in the combiner function). Prior to retrieving a word in a given line, the function validates the word against the words in the STOPWORDS dictionary. If the word exists in the STOPWORDS dictionary, that word is not retrieved. 

**combiner_count_words()** - This function counts the number of time each word has been retrieved. It yields a tuple consisting of an individual word and that word's count. This function is essentially a mini reducer and its use is optional, but it can be useful to use for memory optimization purposes. 

**reducer_count_words()** - This function is the MapReduce reducer, which sums all occurrences of each word and yields a tuple of the form (counts, word). 

**reducer_find_max_word** - This function yields the max word/count pair after the reduce step in the form of a tuple, formatted as (counts, word) using the built in max() function.

**Line 1**:North American plants are plants too

(2, plants)

**Line 2**: North American animals are American

(3, American)

**Line 3**: South American Ecosystems

**(4, American)**

Utilizing these three made up lines of text as a hypothetical example, the job will maintain a running count of the most utilized word. Upon consuming all of the lines, it would finally output the tuple (4, American), since the word American is the most utilized word in the text. 



Now run the file against data/gutenberg/short.t1.txt.

In [5]:
!python most_used_word.py data/gutenberg/short.t1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/most_used_word.jovyan.20200705.224824.038916
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/most_used_word.jovyan.20200705.224824.038916/output
Streaming final output from /tmp/most_used_word.jovyan.20200705.224824.038916/output...
11	"day"
Removing temp directory /tmp/most_used_word.jovyan.20200705.224824.038916...
0.7090287208557129


### <font color="magenta">Q4: Run the above script on the Shakespeare text file.  What answer do you get?</font>

In [6]:
!python most_used_word.py data/gutenberg/t8.shakespeare.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/most_used_word.jovyan.20200705.224824.963658
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/most_used_word.jovyan.20200705.224824.963658/output
Streaming final output from /tmp/most_used_word.jovyan.20200705.224824.963658/output...
5479	"thou"
Removing temp directory /tmp/most_used_word.jovyan.20200705.224824.963658...
4.673604726791382


Upon running the most_used_word.py file against the Shakespeare text file, the most utilized word in the text is "though", with a count of 5479.

### <font color="magenta">Q5: What is the impact of removing the combiner from the above code in terms of efficiency?  What does that suggest?</font>

In [7]:
%%file most_used_word.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+") # any whitespace or apostrophe, used to split lines below


class MRMostUsedWord(MRJob):
    STOPWORDS = {'i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            if word.lower() not in self.STOPWORDS:
                yield (word.lower(), 1)

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is used so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)



if __name__ == '__main__':
    import time
    start = time.time()
    MRMostUsedWord.run()
    end = time.time()
    print(end - start)

Overwriting most_used_word.py


In [8]:
!python most_used_word.py data/gutenberg/t8.shakespeare.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/most_used_word.jovyan.20200705.224829.887801
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/most_used_word.jovyan.20200705.224829.887801/output
Streaming final output from /tmp/most_used_word.jovyan.20200705.224829.887801/output...
5479	"thou"
Removing temp directory /tmp/most_used_word.jovyan.20200705.224829.887801...
3.8563339710235596


Interestingly, by removing the combiner in the most_used_word.py, the amount of time taken to yield the final output is marginally faster (~7.6 vs. ~5.9). This might imply that adding another mini-reducer in the form of the combiner increases the amount of time taken for the job to complete, perhaps because it creates **additional** reduce tasks. 

### <font color="magenta">Q6: Write an mrjob script that finds the 10 words that have the most syllables from the t5.churchill.txt file.  Interpret your results.</font>

In [9]:
%%file most_syllables_top_ten.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
import syllapy



WORD_RE = re.compile(r"[\w']+") # any whitespace or apostrophe, used to split lines below



class MRMostSyllablesTopTen(MRJob):
    
    STOPWORDS = {'i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
  
   
    def steps(self):
        return [MRStep(mapper=self.mapper_get_words), MRStep(reducer = self.reducer)]
  
    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            if word.lower() not in self.STOPWORDS:
                syllable_count =  syllapy.count(word)
                yield None, (syllable_count, word.lower())
        

    def reducer(self, key, values):
        self.alist = []
        for value in values:
            self.alist.append(value)
            self.blist = []
        for i in range(10):
            self.blist.append(max(self.alist))
            self.alist.remove(max(self.alist))
        for i in range(10):
            yield self.blist[i]





if __name__ == '__main__':
    import time
    start = time.time()
    MRMostSyllablesTopTen.run()
    end = time.time()
    print(end - start)

Overwriting most_syllables_top_ten.py


In [10]:
!python most_syllables_top_ten.py data/gutenberg/t5.churchill.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/most_syllables_top_ten.jovyan.20200705.224834.233530
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/most_syllables_top_ten.jovyan.20200705.224834.233530/output
Streaming final output from /tmp/most_syllables_top_ten.jovyan.20200705.224834.233530/output...
8	"overcapitalization"
8	"incommunicability"
7	"unenforceability"
7	"overcapitalized"
7	"materialistically"
7	"invulnerability"
7	"interrogatively"
7	"infinitesimally"
7	"indissolubility"
7	"indispensability"
Removing temp directory /tmp/most_syllables_top_ten.jovyan.20200705.224834.233530...
8.836472511291504


I decided to divide this MapReduce job into two distinct steps(just one mapper and just one reducer) without any intermediary reducing(i.e. combiner). The mapper step retrieves all words in each line of the text, excluding the words specified in the STOPWORDS dictionary, and calculates the syllable count for each word using the external syllapy library. The mapper then yields each word, formatted as a tuple (syllable_count, word.lower()). The sorting happens within the reducer step. All syllable/word pairs are appended to alist. Then, the top ten max values within alist is appended to blist, using a for loop and the built in max() function. Then, all values within blist are appended to the list without_duplictes(duplicates removed using list comprehension). Then, the top ten elements in without_duplicates are yielded using a for loop. Based on the output, the words with the most syllables are 'overcapitalization' and 'incommunicability', at 8 syllables each. 

If the hardcoded range is substantially greater than 10, one starts to run into the issue of duplicates in the output, since, naturally, some words occur in the text more than once. While this is not an inherently bad thing, it results in visual clutter for larger requested outputs. When I attempted to remove duplicates by converting the final list of tuples to a set of tuples, through generic for loops and list comprehension, or through the numpy.unique() function, the operations ended up culling large swaths of entries from the final list, as opposed to only duplicate entries. For the purpose of this specific ask, however, this code does suffice. 